In [107]:
import pandas as pd
import sqlite3
from sqlite3 import connect
from sqlalchemy import create_engine
from datetime import date, datetime
from pytz import timezone
import plotly.express as px
import numpy as np

In [127]:
con = sqlite3.connect('/Users/sean/labs/Capstone/TwitterStockAnalysis/sentiment.db')

In [128]:
df = pd.read_sql_query("SELECT * FROM tsla_stock_prices", con)

In [129]:
df.head()

,index,open,high,low,last,close,volume,date,symbol,exchange
0,0,738.53,760.16,738.14,753.05,733.57,308570.0,2021-09-07T19:30:00+0000,TSLA,IEXG
1,1,738.53,760.16,738.14,754.94,733.57,291181.0,2021-09-07T19:00:00+0000,TSLA,IEXG
2,2,738.53,760.16,738.14,752.89,733.57,274920.0,2021-09-07T18:30:00+0000,TSLA,IEXG
3,3,738.53,760.16,738.14,754.47,733.57,259757.0,2021-09-07T18:00:00+0000,TSLA,IEXG
4,4,738.53,760.16,738.14,752.29,733.57,242599.0,2021-09-07T17:30:00+0000,TSLA,IEXG


In [130]:
df_2_columns = df[['last', 'date']]

In [131]:
df_2_columns.head()

,last,date
0,753.05,2021-09-07T19:30:00+0000
1,754.94,2021-09-07T19:00:00+0000
2,752.89,2021-09-07T18:30:00+0000
3,754.47,2021-09-07T18:00:00+0000
4,752.29,2021-09-07T17:30:00+0000


In [132]:
for index, row in df_2_columns['date'].iteritems():
    df_2_columns.loc[index, 'date'] = row.replace('T', ' ').replace('+', '')

/Users/sean/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [133]:
df_2_columns

,last,date
0,753.05,2021-09-07 19:30:000000
1,754.94,2021-09-07 19:00:000000
2,752.89,2021-09-07 18:30:000000
3,754.47,2021-09-07 18:00:000000
4,752.29,2021-09-07 17:30:000000
...,...,...
95,NaN,2021-08-31 00:00:000000
96,NaN,2021-08-30 23:30:000000
97,NaN,2021-08-30 23:00:000000
98,NaN,2021-08-30 22:30:000000


In [134]:
eastern = timezone('US/Eastern')
utc = timezone('UTC')

# "%d/%m/%Y %H:%M:%S"
for index, row in df_2_columns['date'].iteritems():
    
    created_at = datetime.strptime(row[:-5], '%Y-%m-%d %H:%M:%S')
    utc_created_at = utc.localize(created_at)
    #print(utc_created_at)
    row = utc_created_at.astimezone(eastern)
    df_2_columns.loc[index, 'date_est'] = str(row)

/Users/sean/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/sean/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [135]:
df_2_columns
df_2_columns.drop(columns = 'date')

,last,date_est
0,753.05,2021-09-07 15:30:00-04:00
1,754.94,2021-09-07 15:00:00-04:00
2,752.89,2021-09-07 14:30:00-04:00
3,754.47,2021-09-07 14:00:00-04:00
4,752.29,2021-09-07 13:30:00-04:00
...,...,...
95,NaN,2021-08-30 20:00:00-04:00
96,NaN,2021-08-30 19:30:00-04:00
97,NaN,2021-08-30 19:00:00-04:00
98,NaN,2021-08-30 18:30:00-04:00


In [137]:
for index, row in df_2_columns['date_est'].iteritems():
    #print(row[:10])
    # time is currently utc
    df_2_columns.loc[index, 'time'] = row[11:16]
    df_2_columns.loc[index, 'new_date'] = row[:10]
df_2_columns

,last,date,date_est,time,new_date
0,753.05,2021-09-07 19:30:000000,2021-09-07 15:30:00-04:00,15:30,2021-09-07
1,754.94,2021-09-07 19:00:000000,2021-09-07 15:00:00-04:00,15:00,2021-09-07
2,752.89,2021-09-07 18:30:000000,2021-09-07 14:30:00-04:00,14:30,2021-09-07
3,754.47,2021-09-07 18:00:000000,2021-09-07 14:00:00-04:00,14:00,2021-09-07
4,752.29,2021-09-07 17:30:000000,2021-09-07 13:30:00-04:00,13:30,2021-09-07
...,...,...,...,...,...
95,NaN,2021-08-31 00:00:000000,2021-08-30 20:00:00-04:00,20:00,2021-08-30
96,NaN,2021-08-30 23:30:000000,2021-08-30 19:30:00-04:00,19:30,2021-08-30
97,NaN,2021-08-30 23:00:000000,2021-08-30 19:00:00-04:00,19:00,2021-08-30
98,NaN,2021-08-30 22:30:000000,2021-08-30 18:30:00-04:00,18:30,2021-08-30


In [138]:
df_2_columns = df_2_columns.drop(columns = 'date')

df_2_columns.drop(columns = 'date_est')

,last,time,new_date
0,753.05,15:30,2021-09-07
1,754.94,15:00,2021-09-07
2,752.89,14:30,2021-09-07
3,754.47,14:00,2021-09-07
4,752.29,13:30,2021-09-07
...,...,...,...
95,NaN,20:00,2021-08-30
96,NaN,19:30,2021-08-30
97,NaN,19:00,2021-08-30
98,NaN,18:30,2021-08-30


In [139]:
df_3 = df_2_columns

In [140]:
df_4 = df_3[df_3['new_date'].str.match('2021-09-03')]
df_4.head()

df_4 = df_4.sort_values(by = ['time'])

In [142]:
df_4.head()

,last,date_est,time,new_date
34,734.78,2021-09-03 09:00:00-04:00,09:00,2021-09-03
33,731.61,2021-09-03 09:30:00-04:00,09:30,2021-09-03
32,731.93,2021-09-03 10:00:00-04:00,10:00,2021-09-03
31,730.00,2021-09-03 10:30:00-04:00,10:30,2021-09-03
30,730.18,2021-09-03 11:00:00-04:00,11:00,2021-09-03


In [ ]:
df_5 = pd.DataFrame[['last', tim]]
for index, row in df_4[].iteritems():
    if row == '16:00':
        break
    else:
        df_5 = 

In [116]:
fig = px.line(df_4, x = 'time', y = 'last', title = 'Stock Price on 2021-09-03')
fig.show()